In [3]:
import re
import os
import sys
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from fractions import Fraction
import pandas_helpers
import logging
from logging.config import dictConfig
import config
import nba_py

In [4]:
os.makedirs(config.LOG_DIR, exist_ok=True)
dictConfig(config.LOGGING_CONFIG_DICT)

In [5]:
logging.info("reading attendance data")
attendance = pd.read_excel("../data/raw/nba_team_annual_attendance.xlsx")\
    .clean_column_names()\
    .parse_date_columns()\
    .deduplicate(["team", "starting_year"])


reading attendance data
clean_column_names
parse_date_columns
deduplicate


In [6]:
logging.info("reading player value data")
bestLineups = pd.read_csv('../data/raw/nba_player_value/data/bestLineups.csv')\
                .clean_column_names()\
                .parse_date_columns()


reading player value data
clean_column_names
parse_date_columns


In [7]:
lineups_16_17 = pd.read_csv('../data/raw/nba_player_value/data/lineups_16_17.csv')\
                  .clean_column_names()\
                  .parse_date_columns()


clean_column_names
parse_date_columns


In [8]:
nba_positions = pd.read_csv('../data/raw/nba_player_value/data/nba_positions.csv')\
                  .clean_column_names()\
                  .parse_date_columns()


clean_column_names
parse_date_columns


In [9]:
nba_stats = pd.read_csv('../data/raw/nba_player_value/data/nbaStats.csv')\
    .clean_column_names()\
    .parse_date_columns()


clean_column_names
parse_date_columns


In [10]:
ncaa_positions = pd.read_csv('../data/raw/nba_player_value/data/ncaa_positions.csv')\
    .clean_column_names()\
    .parse_date_columns()


clean_column_names
parse_date_columns


In [11]:
predictions = pd.read_csv('../data/raw/nba_player_value/data/predictions.csv')\
    .clean_column_names()\
    .parse_date_columns()

logging.info("done reading player value data")

clean_column_names
parse_date_columns
done reading player value data


In [12]:
odds = (
    pd.concat(map(pd.read_excel, glob.glob("../data/raw/odds/*.xlsx")))
    .clean_column_names()
    .deduplicate(["team", "date"])
)

clean_column_names
deduplicate


In [13]:
elo = pd.concat(map(pd.read_csv, glob.glob("../data/raw/nba_elo/nbaallelo.csv")))\
    .clean_column_names()\
    .parse_date_columns()\
    .deduplicate(['team_id','opp_id','date_game'])

clean_column_names
parse_date_columns
deduplicate


In [39]:
combined = (
    header_summary.join(productionsummary_summary, rsuffix="prodsum")
    .join(perf_summary, rsuffix="perf")
    .join(fracstage_summary, rsuffix="frac")
    .merge_multi(test_summary, suffixes=("", "test"))
    .merge_multi(completion_summary, suffixes=("", "comp"))
    .merge_multi(production_summary, suffixes=("", "prod"))
)


In [40]:
combined["days_since_completion"] = (
    combined["first_producing_day_of_month"] - combined["completiondate"]
) / np.timedelta64(1, "D")


In [41]:
combined["increment_30days"] = combined["days_since_completion"].apply(
    lambda x: np.floor(x / 30) + 1
)


In [42]:
combined = combined[
    combined["first_producing_day_of_month"] >= combined["completiondate"]
]
combined.shape


(1089884, 287)

In [43]:
combined_subset = combined[combined["increment_30days"].map(
    lambda x: 0 <= x <= 12)]
combined_subset.shape


(107540, 287)

In [44]:
combined_subset = combined_subset.deduplicate(
    key=["api", "increment_30days"]
).deduplicate(key=["api"], override={"oil": "sum"})

combined_subset.shape


(1997, 287)

In [45]:
combined_subset.to_csv("./data/welldb_combined.csv")
